# Tutorial: QUA calibration nodes on a graph

## Motivation and background

This tutorial explains, with a simple toy example, how to run calibrations on a graph using the *QUA Calibration node*
framework. It is based on the code in [hello QUA](https://github.com/qua-platform/qua-libs/tree/main/examples/basics/hello-qua)
in the [qua-libs](https://github.com/qua-platform/qua-libs) project.

The basic idea of the QUA calibration nodes is to allow the quantum engineer or experimentalist to build up the
quantum machine config required to run an accurate and updated QUA program in an iterative way and to break it down
to a manageable series of small and self-contained steps, each within a `QuaCalNode`.

The problem that `QuaCalNodes`solves is the following: in multiple qubit systems the QUA config becomes a large object
that is full of many parameters. These include IF frequencies, waveform samples and mixer correction entries that need to
be kept up do date and well-calibrated. Furthermore, when tuning up the system, we're often in experimentation mode
so we need a framework that is modular and flexible. `QuaCalNodes`, together with the QPU DB, provide a way to build
the QUA config and modify them both automatically and as well as manually when needed.

## Prerequisites and assumptions

This tutorial assumes a working knowledge of QUA (see [qua-libs](https://github.com/qua-platform/qua-libs))
and familiarity with the QPU DB
(see [section 1](https://github.com/entropy-lab/entropy-qpu/blob/main/docs/qpu_db.ipynb) of the tutorial).

## QUA Calibration nodes

### Basic idea

![linear](linear.png)

QUA cal nodes are `PyNodes` which pass the QUA config as input and output. Each QUA cal node performs a calibration
measurement and modifies the QUA config with only the values that were calibrated in this node. If needed, the node
can also add operations and pulses that are required to perform the calibration. This can be seen in the figure above:
We start with a root node, which is a `PyNode` that only passes a bare-bones QUA config (QUA config v0).
The next node, time of flight calibration, is a `QuaCalNode` that measures the time of flight to a readout resonator.
It updates QPU DB by that value and also modifies the QUA config (becoming QUA config v1), which is passed on to the next node.
The next node does the same, and in this manner the config is built up with fresh calibration values, which are concurrently
stored in the QPU DB.

### Anatomy of a single QUA node

![anatomy](singlenode.png)

Let's look more closely at a single `QuaCalNode`, here time of flight calibration. The node contains 3 methods which
are performed sequentially when running the graph:

* `prepare_config()`
* `run()`
* `update_config()`

The first method, `prepare_config()`, is always executed and can be used to add to the config things required for the
calibration, such as readout pulses, integration weights etc.

The second method, `run()`, performs the actual QUA program and the analysis, and modifies the QPU DB accordingly.

The third method, `update_config()`, takes the values from the QPU DB and updates the QUA config accordingly. For
example, here we'd modify the `time_of_flight` field of the resonator.

You may be wondering why `run()` is optional and why we don't just immediately modify TOF in the QUA config. The reason
is that we may want to build up the config without actually running the calibration - for example if decide that TOF
is sufficiently well calibrated and doesn't need to be re-measured. The decision whether to run a node in a graph or
not is determined by the `run_strategy` argument of the entropy `Graph` object.

### Merging configs

![merge](merge.png)

in many cases, a node can depend on more than one ancestor. In this case, the configs will be automatically merged,
as seen above.

NOTE: If ancestor nodes modify the same config fields, the stored value will be **undetermined**!
Please avoid this situation.

### A note on flexibility

The framework of QPU DB and `QuaCalNodes` is very flexibly by design. If you want, you can add arguments
to the class `__init__` method to define run parameters, or, for example, create the full config in the root node
and only modify its values, thus skipping `prepare_config()` entirely.

The recommended way of using QUA cal nodes, and the config, is as shown here in the tutorial but we really value any
feedback and modification suggestions!


## initializing a QPU DB and an entropy DB

Now that we understand the basic principle, let's look at an example.
The following code, similar to [section 1](https://github.com/entropy-lab/entropy-qpu/blob/main/docs/qpu_db.ipynb)
of the tutorial, creates a QPU DB. It also creates an entropy DB and registers the QPU DB as a resource.

In [2]:
# %load_ext autoreload
# %autoreload 2
from entropylab_qpudb import create_new_qpu_database, CalState, QpuDatabaseConnection
from entropylab.instruments.lab_topology import LabResources, ExperimentResources
from entropylab.results_backend.sqlalchemy.db import SqlAlchemyDB

initial_dict = {
    'res1': {
        'TOF': 240,  # an initial guess for TOF
        'f_res': 6e9  # an initial guess for f_res
    },
}
create_new_qpu_database('db1', initial_dict, force_create=True)

in the following lines we register the resource:

In [3]:
entropydb = SqlAlchemyDB('entropy_db.db')
lab_resources = LabResources(entropydb)
lab_resources.register_resource_if_not_exist(
    'qpu_db', QpuDatabaseConnection, ['db1']
)

opening qpu database db1 from commit <timestamp: 05/30/2021 06:36:00, message: initial commit> at index 0


Now we have the databases ready to go and to be used.

## creating the root node

Below we create the root node. We do not (in this case) take values from QPU DB, but we could if we wanted to.

In [4]:
from entropylab import EntropyContext, pynode
from qm.qua import *

from entropylab_qpudb import QuaConfig


@pynode("root", output_vars={"config"})
def root(context: EntropyContext):
    return {"config": QuaConfig({
        "version": 1,
        "controllers": {
            "con1": {
                "type": "opx1",
                "analog_outputs": {
                    1: {"offset": +0.0},
                },
            }
        },
        "elements": {
            "res1": {
                "singleInput": {"port": ("con1", 1)},
                "intermediate_frequency": 6e9, # this is just a guess, will later be modified by the cal node
                "operations": {
                    "playOp": "constPulse",
                },
            },
        }
    })}

# Measuring TOF and updating the QPU DB and config

Below we see a sketch for an implementation of the two nodes we discussed above: time of flight node and res spectroscopy
node.

In [4]:
from entropylab_qpudb import QuaCalNode

class TOFNode(QuaCalNode):
    def prepare_config(self, config: QuaConfig, context: EntropyContext):
        # here we add a measurement pulse
        print('adding a measurement pulse...')

    def run_program(self, config, context: EntropyContext):
        # here we write a QUA program that measures the TOF, and modify QPU DB accordingly
        print('running QUA program and analyzing...')
        with program() as prog:
            # measure TOF
            pass

        # execute and get results... let's say we found that TOF is 252
        print('updating QPU DB....')
        context.get_resource('qpu_db').set('res1', 'TOF', 252)  # here we update the QPU DB
        context.get_resource('qpu_db').commit('after measuring TOF')  # optionally, commit here to persistent storage
        pass

    def update_config(self, config: QuaConfig, context: EntropyContext):
        # below we update the config from the values stored in the QPU DB
        print('updating QUA config from QPU DB values...')
        config['elements']['res1']['time_of_flight'] = context.get_resource('qpu_db').get('res1', 'TOF').value
        pass

class ResSpecNode(QuaCalNode):

    def prepare_config(self, config: QuaConfig, context: EntropyContext):
        print('asserting that the config has the calibrated TOF value...')
        assert config['elements']['res1']['time_of_flight'] == 252  # this is what we expect
        pass

    def run_program(self, config, context: EntropyContext):
        pass

    def update_config(self, config: QuaConfig, context: EntropyContext):
        pass

## Prepare graph experiment

below we can see the syntax for preparing our short calibration experiment

In [5]:
tofnode_res1 = TOFNode(dependency=root, name='time of flight node')
res_spec_node_res1 = ResSpecNode(dependency=tofnode_res1, name='res spec node')

from entropylab import Graph
experiment_resources = ExperimentResources(entropydb)
experiment_resources.import_lab_resource('qpu_db')
calibration_experiment = Graph(experiment_resources, res_spec_node_res1.ancestors())

opening qpu database db1 from commit <timestamp: 05/28/2021 08:18:57, message: initial commit> at index 0


## run the graph experiment

when we run the experiment, we see in the printouts the expected results. Note that the `prepare_config()` and `update_config()` methods of `TOFNode` are called again when running `ResSpecNode`. While this is not the most efficient way to run the code, it is done for robustness reasons and will be corrected later on.

In [6]:
calibration_experiment.run()

2021-05-28 11:18:57,709 - entropy - INFO - Running node <PyNode> root
2021-05-28 11:18:57,716 - entropy - INFO - Running node <TOFNode> time of flight node
adding a measurement pulse...
running QUA program and analyzing...
updating QPU DB....
commiting qpu database db1 with commit <timestamp: 05/28/2021 08:18:57, message: after measuring TOF> at index 1
2021-05-28 11:18:57,724 - entropy - INFO - Running node <ResSpecNode> res spec node
adding a measurement pulse...
updating QUA config from QPU DB values...
asserting that the config has the calibrated TOF value...
2021-05-28 11:18:57,738 - entropy - INFO - Finished entropy experiment execution successfully


## closing the QPU DB

if you get a "can't lock" error on the DB, run this cell to close it and try again.

In [7]:
experiment_resources.get_resource('qpu_db').close()

closing qpu database db1


## Remove DB files

To remove the DB files created in your workspace for the purpose of this demonstration, run this cell:

In [8]:
from glob import glob
import os
for fl in glob("db1*"):
    os.remove(fl)